In [1]:
import pandas as pd
import time

from IPython.display import display
from geopy.geocoders import Nominatim

pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [7]:
data = pd.read_csv( '../Datasets/data-transform.csv' )

In [3]:
# new date format
data['year'] = pd.to_datetime( data['date'] ).dt.strftime( '%Y' )
data['year_week'] = pd.to_datetime( data['date'] ).dt.strftime( '%Y-%U' )


In [4]:
# create is_waterfront row
data['is_waterfront'] = data['waterfront'].apply( lambda x: 'no' if x == 0 else 'yes' )

In [5]:
# create level for each quartile of price
quartile = list( data['price'].quantile([0.25, 0.5, 0.75]) )

data['price_level'] = data['price'].apply( lambda x: 'level_one'   if x <= quartile[0] else
                                                     'level_two'   if ( x > quartile[0] ) & ( x <= quartile[1] ) else
                                                     'level_three' if ( x > quartile[1] ) & ( x <= quartile[2] ) else
                                                     'level_four' )

In [6]:
# create a condition status
condition = list( data['condition'].unique() )
condition.sort()

data['status_condition'] = data['condition'].apply( lambda x: 'bad' if x <= condition[1] else
                                                    'regular' if x == condition[2] else
                                                    'good' if x == condition[3] else 'excelent' )

In [7]:
# create if a house was renovated or not
data['renovated'] = data['yr_renovated'].apply( lambda x: 'no' if x == 0 else 'yes' )

In [8]:
# create a price per m2
data['price/m2'] = data['price']/( data['sqft_lot']*0.0929 )

In [9]:
# create adress data

# create empty rows:
data['road'] = 'NA'
data['house_number'] = 'NA'
data['neighbourhood'] = 'NA'
data['postcode'] = 'NA'
data['city'] = 'NA'
data['state'] = 'NA'

# initialize API Nominatim
geolocator = Nominatim( user_agent='geopiRequest' )

start_time = time.time()

for i in range( len( data ) ):
    print( 'Loop : {}/{}'.format( i, len(data) ) )
    # create query
    query = str( data.loc[i,'lat'] ) + ',' + str( data.loc[i,'long'] )
    
    # API request
    response = geolocator.reverse( query, timeout=None )
    
    # populate data:
    if 'road' in response.raw['address']:
        data.loc[i, 'road'] = response.raw['address']['road']
        
    if 'house_number' in response.raw['address']:
        data.loc[i, 'house_number'] = response.raw['address']['house_number']
    
    if 'neighbourhood' in response.raw['address']:
        data.loc[i, 'neighbourhood'] = response.raw['address']['neighbourhood']
    
    if 'postcode' in response.raw['address']:
        data.loc[i, 'postcode'] = response.raw['address']['postcode']
    
    if 'city' in response.raw['address']:
        data.loc[i, 'city'] = response.raw['address']['city']
    
    if 'state' in response.raw['address']:
        data.loc[i, 'state'] = response.raw['address']['state']
    
time_elapsed = time.time() - start_time
time_elapsed = time_elapsed/3600
display( time_elapsed )

Loop : 0/21612
Loop : 1/21612
Loop : 2/21612
Loop : 3/21612
Loop : 4/21612
Loop : 5/21612
Loop : 6/21612
Loop : 7/21612
Loop : 8/21612
Loop : 9/21612
Loop : 10/21612
Loop : 11/21612
Loop : 12/21612
Loop : 13/21612
Loop : 14/21612
Loop : 15/21612
Loop : 16/21612
Loop : 17/21612
Loop : 18/21612
Loop : 19/21612
Loop : 20/21612
Loop : 21/21612
Loop : 22/21612
Loop : 23/21612
Loop : 24/21612
Loop : 25/21612
Loop : 26/21612
Loop : 27/21612
Loop : 28/21612
Loop : 29/21612
Loop : 30/21612
Loop : 31/21612
Loop : 32/21612
Loop : 33/21612
Loop : 34/21612
Loop : 35/21612
Loop : 36/21612
Loop : 37/21612
Loop : 38/21612
Loop : 39/21612
Loop : 40/21612
Loop : 41/21612
Loop : 42/21612
Loop : 43/21612
Loop : 44/21612
Loop : 45/21612
Loop : 46/21612
Loop : 47/21612
Loop : 48/21612
Loop : 49/21612
Loop : 50/21612
Loop : 51/21612
Loop : 52/21612
Loop : 53/21612
Loop : 54/21612
Loop : 55/21612
Loop : 56/21612
Loop : 57/21612
Loop : 58/21612
Loop : 59/21612
Loop : 60/21612
Loop : 61/21612
Loop : 62/21612
Lo

Loop : 489/21612
Loop : 490/21612
Loop : 491/21612
Loop : 492/21612
Loop : 493/21612
Loop : 494/21612
Loop : 495/21612
Loop : 496/21612
Loop : 497/21612
Loop : 498/21612
Loop : 499/21612
Loop : 500/21612
Loop : 501/21612
Loop : 502/21612
Loop : 503/21612
Loop : 504/21612
Loop : 505/21612
Loop : 506/21612
Loop : 507/21612
Loop : 508/21612
Loop : 509/21612
Loop : 510/21612
Loop : 511/21612
Loop : 512/21612
Loop : 513/21612
Loop : 514/21612
Loop : 515/21612
Loop : 516/21612
Loop : 517/21612
Loop : 518/21612
Loop : 519/21612
Loop : 520/21612
Loop : 521/21612
Loop : 522/21612
Loop : 523/21612
Loop : 524/21612
Loop : 525/21612
Loop : 526/21612
Loop : 527/21612
Loop : 528/21612
Loop : 529/21612
Loop : 530/21612
Loop : 531/21612
Loop : 532/21612
Loop : 533/21612
Loop : 534/21612
Loop : 535/21612
Loop : 536/21612
Loop : 537/21612
Loop : 538/21612
Loop : 539/21612
Loop : 540/21612
Loop : 541/21612
Loop : 542/21612
Loop : 543/21612
Loop : 544/21612
Loop : 545/21612
Loop : 546/21612
Loop : 547/216

Loop : 971/21612
Loop : 972/21612
Loop : 973/21612
Loop : 974/21612
Loop : 975/21612
Loop : 976/21612
Loop : 977/21612
Loop : 978/21612
Loop : 979/21612
Loop : 980/21612
Loop : 981/21612
Loop : 982/21612
Loop : 983/21612
Loop : 984/21612
Loop : 985/21612
Loop : 986/21612
Loop : 987/21612
Loop : 988/21612
Loop : 989/21612
Loop : 990/21612
Loop : 991/21612
Loop : 992/21612
Loop : 993/21612
Loop : 994/21612
Loop : 995/21612
Loop : 996/21612
Loop : 997/21612
Loop : 998/21612
Loop : 999/21612
Loop : 1000/21612
Loop : 1001/21612
Loop : 1002/21612
Loop : 1003/21612
Loop : 1004/21612
Loop : 1005/21612
Loop : 1006/21612
Loop : 1007/21612
Loop : 1008/21612
Loop : 1009/21612
Loop : 1010/21612
Loop : 1011/21612
Loop : 1012/21612
Loop : 1013/21612
Loop : 1014/21612
Loop : 1015/21612
Loop : 1016/21612
Loop : 1017/21612
Loop : 1018/21612
Loop : 1019/21612
Loop : 1020/21612
Loop : 1021/21612
Loop : 1022/21612
Loop : 1023/21612
Loop : 1024/21612
Loop : 1025/21612
Loop : 1026/21612
Loop : 1027/21612
Loo

Loop : 1428/21612
Loop : 1429/21612
Loop : 1430/21612
Loop : 1431/21612
Loop : 1432/21612
Loop : 1433/21612
Loop : 1434/21612
Loop : 1435/21612
Loop : 1436/21612
Loop : 1437/21612
Loop : 1438/21612
Loop : 1439/21612
Loop : 1440/21612
Loop : 1441/21612
Loop : 1442/21612
Loop : 1443/21612
Loop : 1444/21612
Loop : 1445/21612
Loop : 1446/21612
Loop : 1447/21612
Loop : 1448/21612
Loop : 1449/21612
Loop : 1450/21612
Loop : 1451/21612
Loop : 1452/21612
Loop : 1453/21612
Loop : 1454/21612
Loop : 1455/21612
Loop : 1456/21612
Loop : 1457/21612
Loop : 1458/21612
Loop : 1459/21612
Loop : 1460/21612
Loop : 1461/21612
Loop : 1462/21612
Loop : 1463/21612
Loop : 1464/21612
Loop : 1465/21612
Loop : 1466/21612
Loop : 1467/21612
Loop : 1468/21612
Loop : 1469/21612
Loop : 1470/21612
Loop : 1471/21612
Loop : 1472/21612
Loop : 1473/21612
Loop : 1474/21612
Loop : 1475/21612
Loop : 1476/21612
Loop : 1477/21612
Loop : 1478/21612
Loop : 1479/21612
Loop : 1480/21612
Loop : 1481/21612
Loop : 1482/21612
Loop : 148

Loop : 1884/21612
Loop : 1885/21612
Loop : 1886/21612
Loop : 1887/21612
Loop : 1888/21612
Loop : 1889/21612
Loop : 1890/21612
Loop : 1891/21612
Loop : 1892/21612
Loop : 1893/21612
Loop : 1894/21612
Loop : 1895/21612
Loop : 1896/21612
Loop : 1897/21612
Loop : 1898/21612
Loop : 1899/21612
Loop : 1900/21612
Loop : 1901/21612
Loop : 1902/21612
Loop : 1903/21612
Loop : 1904/21612
Loop : 1905/21612
Loop : 1906/21612
Loop : 1907/21612
Loop : 1908/21612
Loop : 1909/21612
Loop : 1910/21612
Loop : 1911/21612
Loop : 1912/21612
Loop : 1913/21612
Loop : 1914/21612
Loop : 1915/21612
Loop : 1916/21612
Loop : 1917/21612
Loop : 1918/21612
Loop : 1919/21612
Loop : 1920/21612
Loop : 1921/21612
Loop : 1922/21612
Loop : 1923/21612
Loop : 1924/21612
Loop : 1925/21612
Loop : 1926/21612
Loop : 1927/21612
Loop : 1928/21612
Loop : 1929/21612
Loop : 1930/21612
Loop : 1931/21612
Loop : 1932/21612
Loop : 1933/21612
Loop : 1934/21612
Loop : 1935/21612
Loop : 1936/21612
Loop : 1937/21612
Loop : 1938/21612
Loop : 193

...

Loop : 20023/21612
Loop : 20024/21612
Loop : 20025/21612
Loop : 20026/21612
Loop : 20027/21612
Loop : 20028/21612
Loop : 20029/21612
Loop : 20030/21612
Loop : 20031/21612
Loop : 20032/21612
Loop : 20033/21612
Loop : 20034/21612
Loop : 20035/21612
Loop : 20036/21612
Loop : 20037/21612
Loop : 20038/21612
Loop : 20039/21612
Loop : 20040/21612
Loop : 20041/21612
Loop : 20042/21612
Loop : 20043/21612
Loop : 20044/21612
Loop : 20045/21612
Loop : 20046/21612
Loop : 20047/21612
Loop : 20048/21612
Loop : 20049/21612
Loop : 20050/21612
Loop : 20051/21612
Loop : 20052/21612
Loop : 20053/21612
Loop : 20054/21612
Loop : 20055/21612
Loop : 20056/21612
Loop : 20057/21612
Loop : 20058/21612
Loop : 20059/21612
Loop : 20060/21612
Loop : 20061/21612
Loop : 20062/21612
Loop : 20063/21612
Loop : 20064/21612
Loop : 20065/21612
Loop : 20066/21612
Loop : 20067/21612
Loop : 20068/21612
Loop : 20069/21612
Loop : 20070/21612
Loop : 20071/21612
Loop : 20072/21612
Loop : 20073/21612
Loop : 20074/21612
Loop : 20075

Loop : 20455/21612
Loop : 20456/21612
Loop : 20457/21612
Loop : 20458/21612
Loop : 20459/21612
Loop : 20460/21612
Loop : 20461/21612
Loop : 20462/21612
Loop : 20463/21612
Loop : 20464/21612
Loop : 20465/21612
Loop : 20466/21612
Loop : 20467/21612
Loop : 20468/21612
Loop : 20469/21612
Loop : 20470/21612
Loop : 20471/21612
Loop : 20472/21612
Loop : 20473/21612
Loop : 20474/21612
Loop : 20475/21612
Loop : 20476/21612
Loop : 20477/21612
Loop : 20478/21612
Loop : 20479/21612
Loop : 20480/21612
Loop : 20481/21612
Loop : 20482/21612
Loop : 20483/21612
Loop : 20484/21612
Loop : 20485/21612
Loop : 20486/21612
Loop : 20487/21612
Loop : 20488/21612
Loop : 20489/21612
Loop : 20490/21612
Loop : 20491/21612
Loop : 20492/21612
Loop : 20493/21612
Loop : 20494/21612
Loop : 20495/21612
Loop : 20496/21612
Loop : 20497/21612
Loop : 20498/21612
Loop : 20499/21612
Loop : 20500/21612
Loop : 20501/21612
Loop : 20502/21612
Loop : 20503/21612
Loop : 20504/21612
Loop : 20505/21612
Loop : 20506/21612
Loop : 20507

Loop : 20887/21612
Loop : 20888/21612
Loop : 20889/21612
Loop : 20890/21612
Loop : 20891/21612
Loop : 20892/21612
Loop : 20893/21612
Loop : 20894/21612
Loop : 20895/21612
Loop : 20896/21612
Loop : 20897/21612
Loop : 20898/21612
Loop : 20899/21612
Loop : 20900/21612
Loop : 20901/21612
Loop : 20902/21612
Loop : 20903/21612
Loop : 20904/21612
Loop : 20905/21612
Loop : 20906/21612
Loop : 20907/21612
Loop : 20908/21612
Loop : 20909/21612
Loop : 20910/21612
Loop : 20911/21612
Loop : 20912/21612
Loop : 20913/21612
Loop : 20914/21612
Loop : 20915/21612
Loop : 20916/21612
Loop : 20917/21612
Loop : 20918/21612
Loop : 20919/21612
Loop : 20920/21612
Loop : 20921/21612
Loop : 20922/21612
Loop : 20923/21612
Loop : 20924/21612
Loop : 20925/21612
Loop : 20926/21612
Loop : 20927/21612
Loop : 20928/21612
Loop : 20929/21612
Loop : 20930/21612
Loop : 20931/21612
Loop : 20932/21612
Loop : 20933/21612
Loop : 20934/21612
Loop : 20935/21612
Loop : 20936/21612
Loop : 20937/21612
Loop : 20938/21612
Loop : 20939

Loop : 21319/21612
Loop : 21320/21612
Loop : 21321/21612
Loop : 21322/21612
Loop : 21323/21612
Loop : 21324/21612
Loop : 21325/21612
Loop : 21326/21612
Loop : 21327/21612
Loop : 21328/21612
Loop : 21329/21612
Loop : 21330/21612
Loop : 21331/21612
Loop : 21332/21612
Loop : 21333/21612
Loop : 21334/21612
Loop : 21335/21612
Loop : 21336/21612
Loop : 21337/21612
Loop : 21338/21612
Loop : 21339/21612
Loop : 21340/21612
Loop : 21341/21612
Loop : 21342/21612
Loop : 21343/21612
Loop : 21344/21612
Loop : 21345/21612
Loop : 21346/21612
Loop : 21347/21612
Loop : 21348/21612
Loop : 21349/21612
Loop : 21350/21612
Loop : 21351/21612
Loop : 21352/21612
Loop : 21353/21612
Loop : 21354/21612
Loop : 21355/21612
Loop : 21356/21612
Loop : 21357/21612
Loop : 21358/21612
Loop : 21359/21612
Loop : 21360/21612
Loop : 21361/21612
Loop : 21362/21612
Loop : 21363/21612
Loop : 21364/21612
Loop : 21365/21612
Loop : 21366/21612
Loop : 21367/21612
Loop : 21368/21612
Loop : 21369/21612
Loop : 21370/21612
Loop : 21371

17395.482928037643

In [10]:
display( data )

,id,date,price,bedrooms,bathrooms,sqft_lot,floors,waterfront,condition,yr_built,...,price_level,status_condition,renovated,price/m2,road,house_number,neighbourhood,postcode,city,state
0,7129300520,20141013T000000,221900.00,3,1.00,5650,1.00,0,3,1955,...,level_one,regular,no,422.76,61st Avenue South,10012,Rainier Beach,98178,Seattle,Washington
1,6414100192,20141209T000000,538000.00,3,2.25,7242,2.00,0,3,1951,...,level_three,regular,yes,799.66,Northeast 127th Street,837,NA,98125,Seattle,Washington
2,5631500400,20150225T000000,180000.00,2,1.00,10000,1.00,0,3,1933,...,level_one,regular,no,193.76,81st Avenue Northeast,15109,NA,98028,NA,Washington
3,2487200875,20141209T000000,604000.00,4,3.00,5000,1.00,0,5,1965,...,level_three,excelent,no,1300.32,Fauntleroy Way Southwest,9236,NA,98136,Seattle,Washington
4,1954400510,20150218T000000,510000.00,3,2.00,8080,1.00,0,3,1987,...,level_three,regular,no,679.43,221st Avenue Northeast,923,NA,98074,NA,Washington
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21607,263000018,20140521T000000,360000.00,3,2.50,1131,3.00,0,3,2009,...,level_two,regular,no,3426.29,North 97th Street,927,Licton Springs,98103,Seattle,Washington
21608,6600060120,20150223T000000,400000.00,4,2.50,5813,2.00,0,3,2014,...,level_two,regular,no,740.70,Southwest 103rd Place,2211,NA,98146,Seattle,Washington
21609,1523300141,20140623T000000,402101.00,2,0.75,1350,2.00,0,3,2009,...,level_two,regular,no,3206.16,26th Avenue South,902,NA,98144,Seattle,Washington
21610,291310100,20150116T000000,400000.00,3,2.50,2388,2.00,0,3,2004,...,level_two,regular,no,1803.06,Northwest Boulder Way Drive,2136,NA,98027,NA,Washington


In [13]:
data.to_csv( '../Datasets/data_feature.csv', index=False )

### Draft

In [30]:
data['coordinates'] = data[['lat', 'long']].apply( lambda x:  str( x['lat'] ) + ',' + str( x['long'] ), axis=1) 

In [ ]:
import aux_functions
from multiprocessing import Pool

df1 = data[['id','coordinates']]

p = Pool( 3 )

start = time.process_time()
df1[['road', 'house_number', 'neighbourhood', 'postcode', 'city', 'town', 'hamlet', 'state']] = p.map( aux_functions.get_geodata, df1.iterrows() )
end = time.process_time()

print( 'Time elapsed: {}', end - start )

In [ ]:
x = 17395.482928037643/3600
print( x )